# 画像認識2：はじめてのNeural Network

深層学習に進む前に、ニューラルネットワークで簡単な分類問題を解いてみましょう。

## 今回挑戦する画像認識タスク

機械学習で最も有名だと言っても過言ではない、アヤメのデータセットを使って分類問題を解いてみましょう。   
これは3種類のアヤメ（'setosa'、'versicolor'、'virginica'）それぞれ50個ずつの個体について、ガクの長さ（sepal length)、ガクの幅（sepal width)、花弁の長さ(petal length)、花弁の幅(petal width)を調べたもので、計150個のサンプルからなり、1つのサンプルには4つの値を持っています。   
その4つの値（ガク・花弁の長さ・幅）から、そのサンプルがどの種類に属するかを分類するという課題です。

# 1. ライブラリのインストール

本プログラムでは、ニューラルネットワーク実装のためのライブラリとして、Kerasを利用しています。   
'keras'と、そのバックエンドである'tensorflow'をインストールします。   

**tensorflowのインストールによる不具合が見られるようです。**
**Colaboratoryの使用を強くお勧めします。**

## 1.1 ローカルPCの場合（非推奨）

**必ず仮想環境を作ってからパッケージをインストールしてください。**   
ガイダンスの環境設定の資料を参照して、ライブラリのインストールをお願いします。
1. Anaconda Navigatorを開く
2. 「Environments」のタブを開き、中央のフレームで「base(root)」とある右側の「▶」をクリックし、"Open Terminal"をクリック
3. コマンドプロンプトから以下の二つのコマンドを実行  

``conda install -c anaconda tensorflow``   
``conda install -c anaconda keras``

**トラブルが起きる場合はColaboratoryをご利用ください。**

## 1.2 Colaboratoryの場合（推奨）
以下のセルを実行してください。   
**このセルはColaboratoryを起動するたびに必要となります**   

In [ ]:
##################################
### Colaboratoryのみ以下を実行 ###
##################################
import sys
if 'google.colab' in sys.modules:
    !pip install tensorflow
    !pip install keras

## ライブラリを読み込み

Kerasから今回使うライブラリをインポートしましょう。

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

# 計算するたびに違う答えにならないよう、ランダムシードを設定する
np.random.seed(seed=0)

# 2. 学習データの準備

## アヤメのデータセットを読み込み
アヤメのデータセットは様々な場面で使われているので、scikit-learnのライブラリの中にあらかじめ含まれています。   
以下のコードで読み込んでください。

In [ ]:
iris = datasets.load_iris()
X= iris.data # 各サンプルについて、ガク・花弁の長さ・幅に関する4つのデータ
Y = iris.target # 各サンプルについて、アヤメの種類に相当するクラス番号
labels = iris.target_names

Yには、それぞれのサンプルにおけるアヤメの種類に対応するクラス番号が記録されています。   
これは、0のとき'setosa'、1のとき'versicolor'、2のとき'virginica'です。

In [ ]:
print('最初の10サンプル：', Y[:10])
print('ラベルの種類：',labels)
print('最初の10サンプルのラベル：', [labels[l] for l in Y[:10]])

Xには、それぞれのサンプルにおけるガクの長さ（sepal length)、ガクの幅（sepal width)、花弁の長さ(petal length)、花弁の幅(petal width)の4つの値が記録されています。   
Xの最初の3個のサンプルを書き出してみましょう。   

In [ ]:
print('Xのサイズ：', X.shape)
for i in range(3):
    print(i,'個目のサンプル：', X[i])

## 正解ラベルをone-hotベクトルに変換
以上でわかるように、`y`は、3種類のアヤメの種類（'setosa'、'versicolor'、'virginica'）に対応する正解ラベルが0, 1, 2のいずれかの整数値で記録されています。   
これを判別するニューラルネットワークを作りたいのですが、そのネットワークの最終層の出力はどのようなものであればいいでしょうか？   

入力したサンプルのデータ（ガクの幅・長さ、花弁の幅・長さ）に対して出力するのは、そのサンプルが0, 1, 2のそれぞれのクラスである尤もらしさ（確率）です。   
たとえば、正しいクラスが"1"である場合、期待される出力は`[0.1 0.8 0.1]`のように（0から数えて）1番目の次元の確率が最も高いベクトル、   
正しいクラスが"2"である場合、期待される出力は`[0 0.1 0.9]`のように、2番目の次元の確率が最も高いベクトルということになります。   

そこで、それに対応するように、正解ラベルもone-hotベクトル（すなわち、正解のクラスだけが1、残りが0のベクトル）に変換しましょう。   

クラスは全部で3クラスですから、3次元のone-hotベクトルに変換します。   
たとえば正解が「1」であるようなサンプルのクラスは"1"ですが、これを1次元のone-hotベクトルに変換すると、(0から数えて）1番目だけが1で残りは0であるような`[0 1 0]`というベクトルに変換されます。   
これを、すべてのサンプル（計150個）について行うので、`y`は150x3の行列になります。

In [ ]:
y = keras.utils.to_categorical(Y)

yのサイズと、最初の10サンプル分の中身を見てみましょう。   
なお、最初の10サンプルのクラス（アヤメの種類）はすべて"0"です。   
（このデータセットは、最初の50サンプルが'setosa'（クラスラベルは"0")、   
次の50サンプルが'versicolor'（クラスラベルは"1"）、最後の50サンプルが'virginica'（クラスラベルは"2"）となっています）

In [ ]:
print('yのサイズ:', y.shape)
print(y[:10])

訓練データと評価データに分けましょう。   
分ける比率は8:2とします。

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

# 3. ネットワークを設計

今回は、4次元の入力データに対し、次のような構造のネットワークを作りたいと思います（インターネットに接続する環境で見てください）。   
<img src='http://www.hal.t.u-tokyo.ac.jp/~yamakata/lecture/mediaproc/mediaproc6/ImageRecognition2.png'></img>   
1層目は入力が4次元、ユニット数３の全結合層（Full connection layer)、活性化関数は'relu'です。   
2層目は入力が3次元、ユニット数が3の全結合層で、その出力を最後に'softmax'関数により確率に変換して出力します。   

これを実装すると以下のようになります。   
なお、上図では左から入力して層を通過していますが、下のコードでは上から順にサンプルが通過していくと考えてください。   

In [ ]:
model = Sequential()

# Full Connection 1 # 入力は4次元、ユニット数は100
model.add(Dense(input_dim=4, units=100))

# 'relu'で活性化
model.add(Activation('relu'))

# Full Connection 2 # 入力は100次元、ユニット数は3
# これが最終層なので、ユニット数はクラス数（=3）と同じである必要がある
model.add(Dense(input_dim=100, units=3))

# 最後の活性化関数は出力を確率にするためsoftmaxを使用
model.add(Activation('softmax'))

ネットワーク構造のサマリを出力してみましょう。

In [ ]:
model.summary()

# 4. 学習
## 最適化手法・損失関数・評価関数の設定
最適化手法を選択します。   
ここではよく使われる'Adam'を使います。   
損失関数は、今回は判別問題（Classification）なので`'categorical_crossentropy'`を指定します。   
もし回帰問題（Regression) ならば、`'mean_squared_error'`や`'mean_absolute_error'`を指定します。   
取り得る選択肢は[Kerasドキュメント](https://keras.io/ja/losses/)を参照してください。   
評価関数は`'accuracy'`としておきます。これは`'categorical_accuracy'`がデフォルト値です。詳しくは[Kerasドキュメント](https://keras.io/ja/metrics/)か[ソース](https://github.com/keras-team/keras/blob/c2e36f369b411ad1d0a40ac096fe35f73b9dffd3/keras/metrics.py)を参照してください。


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

## 学習パラメータ設定

以下のパラメータを設定
*   バッチサイズ：誤差を逆伝搬する際に、サンプルひとつずつ行うのではなく、いくつかのサンプルの誤差をまとめて逆伝搬します。そのときの1まとまりのサンプル数がバッチサイズです。1, 32, 128, 256, 512などが使われます。バッチサイズが大きいほど、特異値の影響を受けにくくなります。
*   エポック数：深層学習では同じ訓練データを何度も使ってパラメタを更新します。ここで指定するのは最大繰り返し回数です


In [ ]:
batch_size = 32 # バッチサイズ
epochs = 50 # エポック数

## 学習

上で設計したネットワークに訓練データを与えてモデルを学習します。   

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

# 5. 評価
## Closed test
学習に使ったサンプルについて、予測精度を計算します。   
このような評価をClosed testと呼びます。   
モデルはこのサンプルを見たことがあるわけなので、精度は高くなるのが一般的です。   

In [ ]:
score = model.evaluate(x_train, y_train, verbose=1)
print('Closed test loss:', score[0])
print('Closed test accuracy:', score[1])

## Open test
次に、学習に使ったサンプルとは別の、評価用について、予測精度を計算します。   
このような評価をOpen testと呼びます。   
これはモデルが見たことのないサンプルなので、これが実用上の精度評価となります。   
Closed testの精度が十分高いのに、Open testの精度が低い場合は、訓練データに対しモデルが過学習を起こしていると考えることができます。

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Open test loss:', score[0])
print('Open test accuracy:', score[1])

# 6. 入力データに対するクラスラベルの予測

## 複数の評価データを一気に評価する

学習したモデルを使って、個々のサンプルのクラスを予測してみましょう。   
その後、正解ラベルとの混同行列を出力してみましょう。    
'versicolor'と'virginica'を時々間違えるようです。

In [ ]:
from sklearn.metrics import confusion_matrix

predict_classes = model.predict_classes( X )
print('Confusion matrix:\n', confusion_matrix(Y, predict_classes))
print('Prediction labels:\n', predict_classes)
print('Ground truth:\n', Y)

## あるサンプルの予測確率を計算する
あるサンプルが'setosa'、'versicolor'、'virginica'のそれぞれのクラスに属する予測確率を出力してみましょう。   
predが予測、GTが正解（Ground Truth）です。ニューラルネットワークは、この二つのベクトルが一致するように学習を行っています。

In [ ]:
predictions = model.predict( X )

# 予測対象とするサンプルのID.ここをいろいろと変えて試してみましょう。
# ID=0～49のとき'setosa'、ID=50～99のとき'versicolor'、ID=100～149のとき'virginica'です
id = 72 

print(id, '個目のサンプルに対する３つの種類の確率')
print('\tsetosa\tversicolor\tvirginica')
print('pred\t{:>.4f}\t{:>.4f}\t{:.4f}'.
      format(predictions[id][0],predictions[id][1],predictions[id][2]))
print('TG\t{:>.4}\t{:>.4}\t{:>.4}'.
      format(y[id][0],y[id][1],y[id][2]))

# 発展的課題
モデルのユニットや構造、学習パラメータを変えて、結果がどのように変わるか試しましょう。